In [1]:
############## PLEASE RUN THIS CELL FIRST! ###################

# import everything and define a test runner function
from importlib import reload
from helper import run
import bloomfilter
import block
import ecc
import helper
import merkleblock
import network
import script
import tx

In [2]:
from helper import hash256
bit_field_size = 10
bit_field = [0] * bit_field_size
h = hash256(b'hello world')
bit = int.from_bytes(h, 'big') % bit_field_size
bit_field[bit] = 1
print(bit_field)

[0, 0, 0, 0, 0, 0, 0, 0, 0, 1]


In [3]:
from helper import hash256
bit_field_size = 10
bit_field = [0] * bit_field_size
for item in (b'hello world', b'goodbye'):
    h = hash256(item)
    bit = int.from_bytes(h, 'big') % bit_field_size
    bit_field[bit] = 1
print(bit_field)

[0, 0, 1, 0, 0, 0, 0, 0, 0, 1]


### Exercise 1

Calculate the Bloom Filter for 'hello world' and 'goodbye' using the `hash160` hash function over a bit field of 10.

In [4]:
# Exercise 1

from helper import hash160

bit_field_size = 10
bit_field = [0] * bit_field_size
items = (b'hello world', b'goodbye')
# loop through each item
for item in items:
    # hash160 the item
    h = hash160(item)
    # interpret hash as a Big-Endian integer and mod by bit_field_size
    idx = int.from_bytes(h,'big')%bit_field_size
    # set that bit in bit_field to 1
    bit_field[idx] = 1
# print the bit_field
print(bit_field)

[1, 1, 0, 0, 0, 0, 0, 0, 0, 0]


In [5]:
from helper import hash256, hash160
bit_field_size = 10
bit_field = [0] * bit_field_size
for item in (b'hello world', b'goodbye'):
    for hash_function in (hash256, hash160):
        h = hash_function(item)
        bit = int.from_bytes(h, 'big') % bit_field_size
        bit_field[bit] = 1
print(bit_field)

[1, 1, 1, 0, 0, 0, 0, 0, 0, 1]


In [6]:
from helper import murmur3
from bloomfilter import BIP37_CONSTANT
field_size = 2
num_functions = 2
tweak = 42
bit_field_size = field_size * 8
bit_field = [0] * bit_field_size
for phrase in (b'hello world', b'goodbye'):
    for i in range(num_functions):
        seed = i * BIP37_CONSTANT + tweak
        h = murmur3(phrase, seed=seed)
        bit = h % bit_field_size
        bit_field[bit] = 1
print(bit_field)

[0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0]


### Exercise 2

Given a Bloom Filter with size=10, function count=5, tweak=99, what are the bytes that are set after adding these items? (use `bit_field_to_bytes` to convert to bytes)

* `b'Hello World'`
* `b'Goodbye!'`

In [7]:
# Exercise 2

from bloomfilter import BloomFilter, BIP37_CONSTANT
from helper import bit_field_to_bytes, murmur3

field_size = 10
function_count = 5
tweak = 99
items = (b'Hello World',  b'Goodbye!')
# calculate the bitfield size
bit_field_size = field_size * 8
# create an empty bit field
bit_field = [0]*bit_field_size
# loop through items
for item in items:
    # loop through function count
    for i in range(function_count):
        # calculate the seed
        seed = i*BIP37_CONSTANT+tweak
        # get the murmur3 hash of the item using the seed
        h = murmur3(item,seed=seed)
        # mod by the bitfield size
        bit = h%bit_field_size
        # set the bit
        bit_field[bit] = 1
# convert the bit field to bytes
print(bit_field_to_bytes(bit_field).hex())
# print the bytes in hex

4000600a080000010940


### Exercise 3

Write the `add` method for `BloomFilter`

#### Make [this test](/edit/code-ch12/bloomfilter.py) pass: `bloomfilter.py:BloomFilterTest:test_add`

In [8]:
# Exercise 3

reload(bloomfilter)
run(bloomfilter.BloomFilterTest("test_add"))

.
----------------------------------------------------------------------
Ran 1 test in 0.001s

OK


### Exercise 4

Write the  `filterload` payload from the `BloomFilter` class.

#### Make [this test](/edit/code-ch12/bloomfilter.py) pass: `bloomfilter.py:BloomFilterTest:test_filterload`

In [9]:
# Exercise 4

reload(bloomfilter)
run(bloomfilter.BloomFilterTest("test_filterload"))

.
----------------------------------------------------------------------
Ran 1 test in 0.001s

OK


### Exercise 5

Write the `serialize` method for the `GetDataMessage` class.

#### Make [this test](/edit/code-ch12/network.py) pass: `network.py:GetDataMessageTest:test_serialize`

In [10]:
# Exercise 5

reload(network)
run(network.GetDataMessageTest("test_serialize"))

.
----------------------------------------------------------------------
Ran 1 test in 0.000s

OK


In [11]:
from bloomfilter import BloomFilter
from helper import decode_base58
from merkleblock import MerkleBlock
from network import FILTERED_BLOCK_DATA_TYPE, GetHeadersMessage, GetDataMessage, HeadersMessage, SimpleNode
from tx import Tx
last_block_hex = '00000000000538d5c2246336644f9a4956551afb44ba47278759ec55ea912e19'
address = 'mwJn1YPMq7y5F8J3LkC5Hxg9PHyZ5K4cFv'
h160 = decode_base58(address)
node = SimpleNode('testnet.programmingbitcoin.com', testnet=True, logging=False)
bf = BloomFilter(size=30, function_count=5, tweak=90210)
bf.add(h160)
node.handshake()
node.send(bf.filterload())
start_block = bytes.fromhex(last_block_hex)
getheaders = GetHeadersMessage(start_block=start_block)
node.send(getheaders)
headers = node.wait_for(HeadersMessage)
getdata = GetDataMessage()
for b in headers.blocks:
    if not b.check_pow():
        raise RuntimeError('proof of work is invalid')
    getdata.add_data(FILTERED_BLOCK_DATA_TYPE, b.hash())
node.send(getdata)
found = False
while not found:
    message = node.wait_for(MerkleBlock, Tx)
    if message.command == b'merkleblock':
        if not message.is_valid():
            raise RuntimeError('invalid merkle proof')
    else:
        for i, tx_out in enumerate(message.tx_outs):
            if tx_out.script_pubkey.address(testnet=True) == address:
                print('found: {}:{}'.format(message.id(), i))
                found = True
                break

OSError: [Errno 51] Network is unreachable

### Exercise 6

Get the current testnet block ID, send yourself some testnet coins, find the UTXO corresponding to the testnet coin _without using a block explorer_, create a transaction using that UTXO as an input and broadcast the `tx` message on the  testnet network.

In [12]:
# Exercise 6

import time

from block import Block
from bloomfilter import BloomFilter
from ecc import PrivateKey
from helper import hash256, little_endian_to_int, encode_varint, read_varint, decode_base58, SIGHASH_ALL
from merkleblock import MerkleBlock
from network import (
    GetDataMessage,
    GetHeadersMessage,
    HeadersMessage,
    NetworkEnvelope,
    SimpleNode,
    TX_DATA_TYPE,
    FILTERED_BLOCK_DATA_TYPE,
)
from script import p2pkh_script, Script
from tx import Tx, TxIn, TxOut

last_block_hex = ''  # FILL THIS IN

secret = little_endian_to_int(hash256(b''))  # FILL THIS IN
private_key = PrivateKey(secret=secret)
addr = private_key.point.address(testnet=True)
h160 = decode_base58(addr)

target_address = 'mwJn1YPMq7y5F8J3LkC5Hxg9PHyZ5K4cFv'
target_h160 = decode_base58(target_address)
target_script = p2pkh_script(target_h160)
fee = 5000  # fee in satoshis


# connect to testnet.programmingbitcoin.com in testnet mode
node = SimpleNode('testnet.programmingbitcoin.com',testnet=True,logging=False)
# create a bloom filter of size 30 and 5 functions. Add a tweak.
bf = BloomFilter(30,5,12211)
# add the h160 to the bloom filter
bf.add(h160)
# complete the handshake
node.handshake()
# load the bloom filter with the filterload command
node.send(bf.filterload())
# set start block to last_block from above
start_block = bytes.fromhex(last_block_hex)
# send a getheaders message with the starting block
getheaders = GetHeadersMessage(start_block=start_block)
node.send(getheaders)
# wait for the headers message
headers = node.wait_for(HeadersMessage)
# store the last block as None
last_block = None
# initialize the GetDataMessage
getdata = GetDataMessage()
# loop through the blocks in the headers
for h in headers.blocks:
    # check that the proof of work on the block is valid
    if not h.check_pow():
        raise RuntimeError("Something wrong with proof of work")
    # check that this block's prev_block is the last block
    if last_block is not None and h.prev_block != last_block:
        raise RuntimeError("Some issue with chain")
    # add a new item to the get_data_message
    # should be FILTERED_BLOCK_DATA_TYPE and block hash
    getdata.add_data(FILTERED_BLOCK_DATA_TYPE,h.hash())
    # set the last block to the current hash
    last_block = h.hash()
# send the getdata message
node.send(getdata)
# initialize prev_tx and prev_index to None
prev_tx,prev_index,prev_amount = None,None,None
# loop while prev_tx is None 
while prev_tx is None:
    # wait for the merkleblock or tx commands
    message = node.wait_for(MerkleBlock,Tx)
    # if we have the merkleblock command
    if message.command == b'merkleblock':
        # check that the MerkleBlock is valid
        if not message.is_valid():
            raise RuntimeError('invalide merkel proof')
    # else we have the tx command
    else:
        # set the tx's testnet to be True
        message.testnet = True
        # loop through the tx outs
        for i, tx_out in enumerate(message.tx_outs):
            # if our output has the same address as our address we found it
            if tx_out.script_pubkey.address(testnet=True) == addr:
                # we found our utxo. set prev_tx, prev_index, and tx
                prev_tx = message.hash()
                prev_index = i
                prev_amount = tx_out.amount
                print('found: {}:{}'.format(prev_tx.hex(), prev_index))
# create the TxIn
tx_in = TxIn(prev_tx, prev_index)
# calculate the output amount (previous amount minus the fee)
output_amount = prev_amount - fee
# create a new TxOut to the target script with the output amount
tx_out = TxOut(output_amount, target_script)
# create a new transaction with the one input and one output
tx_obj = Tx(1, [tx_in], [tx_out], 0, testnet=True)
# sign the only input of the transaction
print(tx_obj.sign_input(0, private_key))
# serialize and hex to see what it looks like
print(tx_obj.serialize().hex())

# send this signed transaction on the network
node.send(tx_obj)
# wait a sec so this message goes through with time.sleep(1) 
time.sleep(1)
# now ask for this transaction from the other node
getdata = GetDataMessage()
# create a GetDataMessage
# ask for our transaction by adding it to the message
getdata.add_data(TX_DATA_TYPE, tx_obj.hash())
# send the message
node.send(getdata)
# now wait for a Tx response
received_tx = node.wait_for(Tx)
# if the received tx has the same id as our tx, we are done!
if received_tx.id() == tx_obj.id():     
    print('success!')

OSError: [Errno 51] Network is unreachable